In [1]:
import heapq
import pandas as pd
import numpy as np
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
data = pd.read_csv("dataset.csv")

In [3]:
X = data.drop(columns=["Tertarik"])
y = data[["Tertarik"]]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [4]:
class KDNode:
    def __init__(self, points, y, left = None, right = None, distance = np.inf):
        self.points = points
        self.y = y
        self.left = left
        self.right = right
        self.distance = distance
    def __lt__(self, other):
        return self.distance < other.distance

In [5]:
class KDTree:
    def __init__(self, k = 2, p = 2):
        self.tree = None
        self.k = k
        self.p = p
    
    def fit(self, X, y):
        self.X = X
        self.y = y

        if isinstance(X, pd.DataFrame):
            X = X.values
        
        if isinstance(y, pd.DataFrame):
            y = y.values

        self.tree = self._construct_tree(np.array(X), np.array(y).reshape(-1), 0)
    
    def _construct_tree(self, points, y, depth):
        if len(points) == 0:
            return None
        
        k = len(points[0])
        axis = depth % k
        
        sort_by_axis = np.argsort(points[:, axis])
        sorted_points = points[sort_by_axis]
        sorted_y =  y[sort_by_axis]
        mid = len(sorted_points) // 2
      
        return KDNode(
            sorted_points[mid],
            sorted_y[mid],
            self._construct_tree(sorted_points[:mid], sorted_y[:mid], depth + 1),
            self._construct_tree(sorted_points[mid + 1:],sorted_y[mid + 1:], depth + 1)
        )
  
    def _knn_search(self, query_point):
        k = len(query_point)
        heap = []
        
        def search(node, depth):
            if node == None:
                return

            nonlocal heap
            
            d = np.linalg.norm(query_point - node.points, ord = self.p)
            node.distance = -d

            if len(heap) < self.k:
                heapq.heappush(heap, node)
            else:
                heapq.heappushpop(heap, node)

            axis = depth % k

            if query_point[axis] < node.points[axis]:
                close, other = node.left, node.right
            else:
                close, other = node.right, node.left

            search(close, depth + 1)

            delta = abs(query_point[axis] - node.points[axis])
            nearest = abs(heap[0].distance)

            if len(heap) < self.k or delta < nearest:
                search(other, depth + 1)

        search(self.tree, 0)
        return heap

    def predict(self, X_test):
        results = []
        if isinstance(X_test, pd.DataFrame):
            X_test = X_test.values
            
        for test in X_test:
            result = self._knn_search(test)
            predict_values = [item.y for item in result]
            counter = Counter(predict_values)
            results.append(counter.most_common(1)[0][0])
            
        return results

In [6]:
kdtree = KDTree(k = 2)
kdtree.fit(X_train, y_train)

In [7]:
y_pred = kdtree.predict(X_test)

In [8]:
precision = accuracy_score(y_pred, y_test) * 100
print("Accuracy: {0:.2f}%".format(precision))

Accuracy: 89.61%


In [ ]:
from sklearn.neighbors import KNeighborsClassifier

kdtree2 = KNeighborsClassifier(n_neighbors = 2)
kdtree2.fit(X_train, y_train)
y_pred2 = kdtree2.predict(X_test)

In [11]:
precision2 = accuracy_score(y_pred, y_test) * 100
print("Accuracy: {0:.2f}%".format(precision2))

Accuracy: 89.61%
